In [1]:
import time

def print_numbers():
    for i in range(5):
        time.sleep(1)
        print(i)

def print_alphabets():
    for i in ['a', 'b', 'c', 'd', 'e']:
        time.sleep(1)
        print(i)

start_time = time.time()
print_numbers()
print_alphabets()
end_time = time.time()
total_time = end_time - start_time
print(f"Total execution time: {total_time:.2f} seconds")

0
1
2
3
4
a
b
c
d
e
Total execution time: 10.01 seconds


⛔ t1.join()
This tells the main thread (your Python script) to pause and wait until Thread-1 is done.

It doesn’t stop Thread-2. t2 can continue running in parallel.

⛔ t2.join()
After Thread-1 is done, this tells the main thread to now wait for Thread-2 to finish (if it's not done already).

In [6]:
import threading
import time

def print_numbers():
    for i in range(5):
        time.sleep(2)
        print(i)

def print_alphabets():
    for i in ['a', 'b', 'c', 'd', 'e']:
        time.sleep(1)
        print(i)
            
        
# Create threads
thread1 = threading.Thread(target=print_numbers, name="Thread-1")
thread2 = threading.Thread(target=print_alphabets, name="Thread-2")

start_time = time.time()

# Start threads
thread1.start()
thread2.start()

# Wait for threads to finish
thread1.join()
thread2.join()


end_time = time.time()
total_time = end_time - start_time

print(f"woh Total execution time: {total_time:.2f} seconds")

a
0
b
c
1
d
e
2
3
4
woh Total execution time: 10.01 seconds


| Thread Type           | Description                                                                     |
| --------------------- | ------------------------------------------------------------------------------- |
| **Daemon Thread**     | Background thread that is killed when the main thread ends. Optional task.      |
| **Non-Daemon Thread** | Normal thread that keeps running and **blocks** program exit until it finishes. |


| Feature / Aspect        | **CPU-bound Tasks**                                                 | **I/O-bound Tasks**                                 |
| ----------------------- | ------------------------------------------------------------------- | --------------------------------------------------- |
| **Definition**          | Tasks that use CPU heavily for computation                          | Tasks that spend time waiting for input/output      |
| **Main Bottleneck**     | CPU (processing power)                                              | I/O (disk, network, DB, etc.)                       |
| **Examples**            | - Complex math<br>- Image/video processing<br>- Sorting large lists | - File read/write<br>- Web scraping<br>- DB queries |
| **GIL Impact**          | Major limitation in Python with threads                             | Less impact, threads can still switch on wait       |
| **Best Tool**           | ✅ `multiprocessing` or `ProcessPoolExecutor`                        | ✅ `threading` or `ThreadPoolExecutor`               |
| **Runs in Parallel?**   | ✅ Yes (via multiple processes)                                      | ❌ Not true parallelism (concurrent via threads)     |
| **Runs Concurrently?**  | ❌ Not effectively with threads (GIL)                                | ✅ Yes, threads release GIL during I/O               |
| **Use Multiple Cores?** | ✅ Yes (processes can use all CPU cores)                             | ❌ No (threads are limited by GIL)                   |
| **Ideal Use Case**      | Heavy computation tasks                                             | Waiting on I/O or external response                 |


# ThreadPoolExecutor 
- The ThreadPoolExecutor (from concurrent.futures) is a high-level API that makes managing threads super simple, compared to manually starting and joining them.

✅ Why use ThreadPoolExecutor?
Instead of:

- creating Thread() objects,

- manually .start()ing them,

- and calling .join()…

- You can just submit tasks to a thread pool, and Python will handle the rest.

- 🔒 1. A Thread Owns a Task Completely
- That whole function is assigned to one thread Other threads will not “jump in” or “continue from where it paused”.




In [10]:
from concurrent.futures import ThreadPoolExecutor
import time

def print_numbers(n1,n2):
    for i in range(n1+n2):
        time.sleep(1)
        print(i)

def print_alphabets():
    for i in ['a', 'b', 'c', 'd', 'e']:
        time.sleep(1)
        print(i)

start_time = time.time()

with ThreadPoolExecutor(max_workers=2) as executor:
    executor.submit(print_numbers, 5,5)

    executor.submit(print_alphabets)
        

end_time = time.time()
print(f"Total execution time: {end_time - start_time:.2f} seconds")



0
a
1
b
2
c
3
d
4
e
5
6
7
8
9
Total execution time: 10.01 seconds


## ❓ What Is a Deadlock?
- A deadlock happens when two or more threads wait for each other to release resources — but none ever do, so they all get stuck.
-🧠 What is a Lock?
-vA Lock in Python (from the threading module) is a mechanism to prevent multiple threads from accessing shared resources at the same time.
- Think of a Lock like a door lock:
- Only one thread can hold the lock (open the door).
- Others must wait until the lock is released.
## “If Python threads don’t run in parallel due to the GIL, why do we even need locks?”

** Python Threads & the GIL
- CPython (the default Python interpreter) uses a Global Interpreter Lock (GIL).
- The GIL means: only one thread executes Python bytecode at a time, even if you have multiple cores.
- So yes — Python threads don't do true parallelism for CPU-bound work. They do concurrency, not parallel execution.



## So Why Use Locks?
Because even concurrent access to shared variables can be unsafe.

## GIL ≠ Thread-Safety
- Even though only one thread runs at a time in the Python VM:
- Python can switch threads in the middle of an operation.
- This can break multi-step operations like counter += 1, which is:
- Read counter
- Add 1
- Write back
- These steps can be interrupted between, leading to a race condition.

In [1]:
import threading

counter = 0
lock = threading.Lock()  # Just to ensure clean output (optional)

def increment():
    global counter
    for _ in range(10):
        with lock:
            counter += 1
            # Print the thread currently executing
            print(f"{threading.current_thread().name} incremented counter to {counter}")

# Create 2 threads with names
threads = [
    threading.Thread(target=increment, name="Thread-A"),
    threading.Thread(target=increment, name="Thread-B")
]

for t in threads:
    t.start()

for t in threads:
    t.join()

print("Final counter:", counter)


Thread-A incremented counter to 1
Thread-A incremented counter to 2
Thread-A incremented counter to 3
Thread-A incremented counter to 4
Thread-A incremented counter to 5
Thread-A incremented counter to 6
Thread-A incremented counter to 7
Thread-A incremented counter to 8
Thread-A incremented counter to 9
Thread-A incremented counter to 10
Thread-B incremented counter to 11
Thread-B incremented counter to 12
Thread-B incremented counter to 13
Thread-B incremented counter to 14
Thread-B incremented counter to 15
Thread-B incremented counter to 16
Thread-B incremented counter to 17
Thread-B incremented counter to 18
Thread-B incremented counter to 19
Thread-B incremented counter to 20
Final counter: 20


# Output Example (shortened):

Thread-A incremented counter to 1  
Thread-B incremented counter to 2  
Thread-A incremented counter to 3  
...
Final counter: 2000

- When we have shared memory data its not safe to move on without lock because in python we cannot assume if thread 1 one is running it will run till end untill its task is done i between it might pause and other thread might come into action 

# 🧠 Real-world Analogy:
- Think of two people at a narrow door.
- Person A is waiting for Person B to move.
- Person B is also waiting for Person A to move.
- So... both are stuck. 🧍‍♂️🚪🧍‍♀️

In [2]:
# Do not run the code for deadlock example as it will cause the program to hang indefinitely.

In [ ]:
import threading
import time

lock1 = threading.Lock()
lock2 = threading.Lock()

def thread1():
    with lock1:
        print("Thread 1 acquired lock1")
        time.sleep(1)
        with lock2:
            print("Thread 1 acquired lock2")

def thread2():
    with lock2:
        print("Thread 2 acquired lock2")
        time.sleep(1)
        with lock1:
            print("Thread 2 acquired lock1")

t1 = threading.Thread(target=thread1)
t2 = threading.Thread(target=thread2)

t1.start()
t2.start()

t1.join()
t2.join()

print("Done")


| Thread 1                         | Thread 2                         |
| -------------------------------- | -------------------------------- |
| Gets `lock1` ✅                   | Gets `lock2` ✅                   |
| Waits for `lock2` ❌ (held by T2) | Waits for `lock1` ❌ (held by T1) |
| Both are stuck forever 😵        | Both are stuck forever 😵        |

Thread 1 acquired lock1  
Thread 2 acquired lock2  
And Your program hangsssssssssssss

In [3]:
# Preventing Deadlocks with Consistent Locking Order

- When Thread 1 acquires lock1 and then acquires lock2, it is holding both locks at the same time — it has not released lock1 while holding lock2.

#### Here’s the step-by-step:

- Thread 1 calls with lock1: — it acquires lock1.

- Inside that block, Thread 1 calls with lock2: — it acquires lock2 while still holding lock1.

- Now Thread 1 has both lock1 and lock2 locked simultaneously.

- It can only release lock2 when the inner with lock2: block ends.

- It can only release lock1 when the outer with lock1: block ends.

In [ ]:
import threading
import time

# Create two locks
lock1 = threading.Lock()
lock2 = threading.Lock()

def thread1():
    print("Thread 1 trying to acquire lock1")
    with lock1:
        print("Thread 1 acquired lock1")
        time.sleep(1)
        print("Thread 1 trying to acquire lock2")
        with lock2:
            print("Thread 1 acquired lock2")
            time.sleep(1)
            print("Thread 1 is doing work")
        print("Thread 1 released lock2")
    print("Thread 1 released lock1")

def thread2():
    print("Thread 2 trying to acquire lock1")
    with lock1:  # 🔑 Same order as thread1
        print("Thread 2 acquired lock1")
        time.sleep(1)
        print("Thread 2 trying to acquire lock2")
        with lock2:
            print("Thread 2 acquired lock2")
            print("Thread 2 is doing work")
            
        print("Thread 2 released lock2")
    print("Thread 2 released lock1")

# Create and start threads
t1 = threading.Thread(target=thread1)
t2 = threading.Thread(target=thread2)

t1.start()
t2.start()

t1.join()
t2.join()

print("✅ Both threads finished without deadlock")


Thread 1 trying to acquire lock1
Thread 1 acquired lock1
Thread 2 trying to acquire lock1
Thread 1 trying to acquire lock2
Thread 1 acquired lock2
Thread 1 is doing work
Thread 1 released lock2
Thread 1 released lock1
Thread 2 acquired lock1
Thread 2 trying to acquire lock2
Thread 2 acquired lock2
Thread 2 is doing work
Thread 2 released lock2
Thread 2 released lock1
✅ Both threads finished without deadlock


# ✅ 11. Producer–Consumer Problem (Inter-Thread Communication)
# 🧠 Concept:
- Producer: Generates data and adds to a shared queue.
- Consumer: Takes data from the queue and processes it.
- queue.Queue() is thread-safe, so no need for manual locking!

In [4]:
import threading
import time
import queue

q = queue.Queue()

def producer():
    for i in range(5):
        print(f"Producer: produced item {i}")
        q.put(i)
        time.sleep(2)
    q.put(None)  # Sentinel to stop consumer

def consumer():
    while True:
        item = q.get()
        if item is None:
            print("Consumer: received None, exiting.")
            break
        print(f"Consumer: consumed item {item}")
        time.sleep(1)

t1 = threading.Thread(target=producer)
t2 = threading.Thread(target=consumer)

t1.start()
t2.start()

t1.join()
t2.join()


Producer: produced item 0
Consumer: consumed item 0
Producer: produced item 1
Consumer: consumed item 1
Producer: produced item 2
Consumer: consumed item 2
Producer: produced item 3
Consumer: consumed item 3
Producer: produced item 4
Consumer: consumed item 4
Consumer: received None, exiting.


In [14]:
import threading
import time

def cpu_bound():
    for i in range(10**8):
        pass
    print("Done")

            
        
# Create threads
thread1 = threading.Thread(target=cpu_bound, name="Thread-1")
thread2 = threading.Thread(target=cpu_bound, name="Thread-2")

start_time = time.time()

# Start threads
thread1.start()
thread2.start()

# Wait for threads to finish
thread1.join()
thread2.join()


end_time = time.time()
total_time = end_time - start_time

print(f"Total execution time: {total_time:.2f} seconds")

Done
Done
Total execution time: 5.81 seconds
